In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

mpl.style.use('ggplot')
plt.rcParams['savefig.dpi'] = 300 #图片像素
plt.rcParams['figure.dpi'] = 300 #分辨率
plt.rc('figure', figsize=(14, 10))#设置图片大小
sns.set_style('white')

%matplotlib inline
%run Metric.ipynb
%run Util.ipynb
%run ChanBiManager.ipynb
%run BiInSectionService.ipynb

fig_size=(12,8)

class KPainter:
    
    def __init__(self,symbol,level,short,mid):
        self.symbol = symbol
        self.level = level
        self.short= short
        self.mid = mid
        
        self.biInSectionService = BiInSectionService(symbol, level, short, mid)
        self.arrow_props = dict(arrowstyle='->', connectionstyle="arc3,rad=.2")
    
    
    def get_xy_coord(self,chan_bi_df,key,anchor_price):
        y = round(100*(chan_bi_df.loc[key,'price']-anchor_price)/anchor_price,2)
        return (chan_bi_df.loc[key,'datetime'], y)
        
    
    def get_xytext_coord(self,row_series):
        if(row_series['turning_point']=='top'):
            return (10, -10)
        else:
            return (-20, -10)
        
        
    def get_the_first_bottom(self,chan_bi_df):
        down_chan_bi_df = chan_bi_df[chan_bi_df['turning_point']=='bottom']
        
        if(down_chan_bi_df.shape[0]>1):
            the_first_bottom = down_chan_bi_df.iloc[0]
                
            return the_first_bottom
        
        
    def get_y_ax_price_increase(self,price,anchor_price):
        return round(100*(price-anchor_price)/anchor_price,2)

    
    def draw_exit_k(self,section_end,anchor_price,chan_bi_df):
        exist_x = to_datetime(section_end) 
        max_price=max(chan_bi_df['price'])
        min_price=min(chan_bi_df['price'])
        
        min_y = self.get_y_ax_price_increase(min_price,anchor_price)
        max_y = self.get_y_ax_price_increase(max_price,anchor_price)
        
        plt.vlines(exist_x, min_y, max_y, colors = "r", linestyles = "dashed")
        
    def draw_enter_k(self,section_start,anchor_price,chan_bi_df):
        enter_x = to_datetime(section_start) 
        max_price=max(chan_bi_df['price'])
        min_price=min(chan_bi_df['price'])
        
        min_y = self.get_y_ax_price_increase(min_price,anchor_price)
        max_y = self.get_y_ax_price_increase(max_price,anchor_price)
        
        plt.vlines(enter_x, min_y, max_y, colors = "r", linestyles = "dashed")

        
    def draw_chan_bi(self,anchor_price,chan_bi_df):
        x_ax = chan_bi_df['datetime']
        y_ax = self.get_y_ax_price_increase(chan_bi_df['price'],anchor_price)
        plt.plot(x_ax,y_ax,'m-o',label='笔')
        plt.grid(axis="y")
        
    def draw_first_bottom(self,anchor_price,chan_bi_df):
        the_first_bi = chan_bi_df.iloc[0]
        
        if(the_first_bi['turning_point']=='bottom'):
            plt.axhline(y=0, color='g', linestyle='-')
        else:
            the_first_bottom = self.get_the_first_bottom(chan_bi_df)
            if(the_first_bottom is not None):
                the_first_bottom_price = round(100*(the_first_bottom['price']-anchor_price)/anchor_price,2)
                plt.axhline(y=the_first_bottom_price, color='g', linestyle='-')

            
    def draw_price_increase_per_anotations(self,anchor_price,chan_bi_df):
        for key,row in chan_bi_df.iterrows():
            increase_per = chan_bi_df.loc[key,'increase']
            
            xy_coord = self.get_xy_coord(chan_bi_df,key,anchor_price)
            xytext_coord = self.get_xytext_coord(row)
            
            plt.annotate(s=str(increase_per)+'%', xy=xy_coord, 
                         xycoords='data', xytext=xytext_coord,
                         textcoords='offset points', fontsize=12,
                         arrowprops=self.arrow_props)
        
        
    def draw_out_area(self,section_start,section_end,chan_bi_df):
        plt.axvspan(chan_bi_df.iloc[0]['datetime'], to_datetime(section_start), color='0.5', alpha=0.2, lw=0)
        plt.axvspan(chan_bi_df.iloc[-1]['datetime'], to_datetime(section_end), color='0.5', alpha=0.2, lw=0)
    
        
#   画一个波段的分笔走势
    def draw_single_level_chan_bi(self,section_start,section_end,start_price,end_price):
        chan_bi_df = self.biInSectionService.get_chan_bi_df_in_section(section_start, section_end)
        if(chan_bi_df.shape[0]==0):
            return

        anchor_price = start_price
        
        plt.rc('figure', figsize=fig_size)
        title_str = self.symbol+' '+self.level+' section ['+section_start+','+section_end+'] MA('+str(self.short)+','+str(self.mid)+') '+chan_bi_df.iloc[0]['date']
        plt.title(title_str)
        
        self.draw_chan_bi(anchor_price,chan_bi_df)
#         self.draw_enter_k(section_start,anchor_price,chan_bi_df)
#         self.draw_exit_k(section_end,anchor_price,chan_bi_df)
        
        self.draw_first_bottom(anchor_price,chan_bi_df)
        self.draw_price_increase_per_anotations(anchor_price,chan_bi_df)
        
        self.draw_out_area(section_start,section_end,chan_bi_df)
  
        fig = plt.gcf()
        fig.set_size_inches(12,8)
        
        file_path = section_fig_dir +'fig_'+self.symbol+'_'+section_start+'_'+section_end+'_'+self.level+'_'+section_version+'.png'
        fig.savefig(file_path, format='png', transparent=True, dpi=300, pad_inches = 0)
        plt.show()
        
